In [1]:
#pip install --upgrade pip


In [2]:
#pip install html5lib

In [3]:
#pip install selenium

In [4]:
#pip install bs4

In [5]:
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By



In [1]:


driver = webdriver.Chrome()
driver.get("https://www.foxnews.com/category/shows/tucker-carlson-tonight/transcript")
button = driver.find_element(By.XPATH, "//div[@class='button load-more js-load-more']") 
x = 0
for x in range(142):
    button.click()    
    time.sleep(1)
    x+=1
    #print(x)
page_source = driver.page_source # save the source to feed to beautiful soup 
print("saved")


KeyboardInterrupt



In [7]:
soup = BeautifulSoup(page_source)


In [8]:
url_soup = soup.find_all('a', href=re.compile('/opinion/.*'))

In [10]:
url_list = []
prefix = 'https://www.foxnews.com/'
for link in url_soup: 
    suffix = re.findall('\"\/(.*?)\"', str(link))
    if suffix:
        url_list.append(prefix + suffix[0])
    

In [17]:
url_list  = [*set(url_list)] 

In [34]:
# verifying for one instance
url = url_list[100]
#url = 'https://www.foxnews.com/opinion/tucker-carlson-faa-travel-pete-buttigieg'
response = requests.get(url)
soup = BeautifulSoup(response.content)
title = soup.title.string

a_tag = soup.find_all('a')
for tag in a_tag:
    tag.decompose()

copy_tag = soup.find_all('p', class_ = "copyright")
for tag in copy_tag:
    tag.decompose()
    
span_tag = soup.find_all('span')
for tag in span_tag:
    tag.decompose()
    
bold_tag = soup.find_all('strong') 
for tag in bold_tag:
    tag.decompose()
    
italic_tag=soup.find_all('i') 
for tag in italic_tag:
    tag.decompose()

other_tag = soup.find_all('p', class_ ="dek")
for tag in other_tag:
    tag.decompose()
    
other_tag2 = soup.find_all('p', class_ ="subscribed hide")
for tag in other_tag2:
    tag.decompose()
    
other_tag3 = soup.find_all('p', class_ ="success hide") 
for tag in other_tag3:
    tag.decompose()



final = soup.find_all('p')

finaltext = ""
for p in final:

    try:
        if len(p.text) > 1:
            finaltext = finaltext + p.text + '\n'+ '\n'
            finaltext = finaltext.replace('\xa0', ' ')

    except:
        continue
print(finaltext)


Fox News host gives his take on pro-abortion protesters targeting Supreme Court justices over the possible overturn of Roe v. Wade on 'Tucker Carlson Tonight.'

It's pretty hard to argue with people who are passive-aggressive. You may have tried it before. "Why are you so angry?" they scream. "Stop being violent," they snarl as they punch you in the face. Passive-aggressive people are intent on dominating you, but they're too dishonest to admit it.  

Now, it's not an honorable style of attack, but it's very effective, mostly because it's so bewildering. The Democratic Party practices this. Democrats will never meet you on an open field of battle. Instead, they will sneak up behind you and knock you unconscious with a bag of sanctimony. This is the party of weak men and angry women, so passive aggression is their only mode of communication. If you've ever seen one of Jen Psaki's press conferences, you know exactly what we're talking about. 

We watched one of them yesterday, in fact. I

In [37]:

full_df = None
dfs = []

for n, url in enumerate(url_list):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    time = soup.find('time').text
    title = soup.title.string

    a_tag = soup.find_all('a')
    for tag in a_tag:
        tag.decompose()

    copy_tag = soup.find_all('p', class_ = "copyright")
    for tag in copy_tag:
        tag.decompose()

    span_tag = soup.find_all('span')
    for tag in span_tag:
        tag.decompose()

    bold_tag = soup.find_all('strong') 
    for tag in bold_tag:
        tag.decompose()

    italic_tag=soup.find_all('i') 
    for tag in italic_tag:
        tag.decompose()

    other_tag = soup.find_all('p', class_ ="dek")
    for tag in other_tag:
        tag.decompose()

    other_tag2 = soup.find_all('p', class_ ="subscribed hide")
    for tag in other_tag2:
        tag.decompose()

    other_tag3 = soup.find_all('p', class_ ="success hide") 
    for tag in other_tag3:
        tag.decompose()


    final = soup.find_all('p')
    # text=''
    # print(final)
    finaltext = ''
    for p in final:
        try:
            if len(p.text) > 1:
                finaltext = finaltext + p.text + '\n'+ '\n'
                finaltext = finaltext.replace('\xa0', ' ')

        except:
            continue
    
    data={'url': [url],'timestamp': [time], 'title': [title], 'transcript': [finaltext]}
    df = pd.DataFrame(data)
    dfs.append(df)

full_df = pd.concat(dfs).reset_index(drop=True)

In [49]:
full_df['timestamp'] = pd.to_datetime(full_df['timestamp'])

C:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
C:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [53]:
full_df=full_df.sort_values(by=['timestamp']).reset_index(drop=True)

In [56]:
full_df.to_csv('Tucker_transcripts.tsv', sep="\t")